<a href="https://colab.research.google.com/github/cs16677/ce888/blob/master/lab5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
import numpy as np
import pandas as pd

df = pd.read_csv('jester-data-1.csv', header=None)
arr = df.values
print(df.head())

   0      1      2      3      4    ...    96     97     98     99     100
0   74  -7.82   8.79  -9.66  -8.16  ...  99.00  -5.63  99.00  99.00  99.00
1  100   4.08  -0.29   6.36   4.37  ...  -2.14   3.06   0.34  -4.32   1.07
2   49  99.00  99.00  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
3   48  99.00   8.35  99.00  99.00  ...  99.00  99.00  99.00  99.00  99.00
4   91   8.50   4.61  -4.17  -5.39  ...   1.55   3.11   6.55   1.80   1.60

[5 rows x 101 columns]


In [0]:
# taken from "albanda/CE888/lab5-recommender/helper_function.ipynb"
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
val_set, idx = replace(arr)

In [0]:
# taken from "albanda/CE888/lab5-recommender/rec_latent.ipynb"

def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations=30000):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if not rating == '99':
                    err = train(user_id, item_id, rating, 0.0001)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()
        print(mse)

In [70]:
n_latent_factors = 1

user_ratings = arr
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))
# stochastic gradient descent
sgd(10)

864.2733176978104
657.1412172854143
605.2166708950839
586.3469228435248
587.4236577530113
581.3075133502641
575.9234583492004
578.6785368119056
596.1772020757196


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in subtract
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in multiply


nan


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: overflow encountered in square
